In [59]:
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine
import pandas as pd
import csv

In [85]:
#
params = {
    'host': '34.209.37.77',
    'user': 'ubuntu',
    'port': 5432
}


In [138]:
connection = connect(**params, dbname='baseball')
cursor = connection.cursor()

1. What was the total spent on salaries by each team, each year?

In [30]:
cursor.execute('''SELECT allstarfull.teamid, allstarfull.yearid, SUM(salaries.salary) as team_sum_by_yea, salaries.teamid
FROM allstarfull
JOIN salaries
ON allstarfull.teamid = salaries.teamid
GROUP BY allstarfull.yearid, allstarfull.teamid, salaries.teamid
ORDER BY allstarfull.yearid;''')


In [31]:
cursor.fetchall()



[('NYA', 1933, 18513452562.0, 'NYA'),
 ('DET', 1933, 1618342187.0, 'DET'),
 ('CHA', 1933, 3236069286.0, 'CHA'),
 ('CLE', 1933, 3968592687.0, 'CLE'),
 ('PHI', 1933, 3602821666.0, 'PHI'),
 ('SLN', 1933, 6619753328.0, 'SLN'),
 ('CIN', 1933, 1326557848.0, 'CIN'),
 ('BOS', 1933, 2293681006.0, 'BOS'),
 ('PIT', 1933, 1834738436.0, 'PIT'),
 ('CHN', 1933, 5382932445.0, 'CHN'),
 ('DET', 1934, 4855026561.0, 'DET'),
 ('CLE', 1934, 2645728458.0, 'CLE'),
 ('SLN', 1934, 6619753328.0, 'SLN'),
 ('BOS', 1934, 2293681006.0, 'BOS'),
 ('PIT', 1934, 2752107654.0, 'PIT'),
 ('CHA', 1934, 3236069286.0, 'CHA'),
 ('CHN', 1934, 8971554075.0, 'CHN'),
 ('NYA', 1934, 18513452562.0, 'NYA'),
 ('CHA', 1935, 1618034643.0, 'CHA'),
 ('PHI', 1935, 1801410833.0, 'PHI'),
 ('CLE', 1935, 3968592687.0, 'CLE'),
 ('SLN', 1935, 11584568324.0, 'SLN'),
 ('CIN', 1935, 1326557848.0, 'CIN'),
 ('DET', 1935, 6473368748.0, 'DET'),
 ('NYA', 1935, 9256726281.0, 'NYA'),
 ('BOS', 1935, 6881043018.0, 'BOS'),
 ('CHN', 1935, 3588621630.0, 'CHN')

2. What is the first and last year played for each player? *Hint:* Create a new table from 'Fielding.csv'.

In [82]:
cursor.execute("""CREATE TABLE IF NOT EXISTS Fieldin (
playerID varchar(20) NOT NULL,
yearID int NOT NULL,
stint varchar(20) NOT NULL,
teamID text NOT NULL,
lgID text DEFAULT NULL,
POS text NOT NULL,
G int DEFAULT NULL,
GS int DEFAULT NULL,
InnOuts int DEFAULT NULL,
PO int DEFAULT NULL,
A  int DEFAULT NULL,
E  int DEFAULT NULL,
DP int DEFAULT NULL,
PB int DEFAULT NULL,
WP int DEFAULT NULL,
SB int DEFAULT NULL,
CS int DEFAULT NULL,
ZR int DEFAULT NULL,
PRIMARY KEY (playerID,yearID, stint, teamID, pos)
);""")


In [ ]:
cursor.execute("COPY Fieldin FROM '/home/ubuntu/baseballdata/Fielding.csv' DELIMITER ',' CSV HEADER;")
connection.commit()

In [87]:
cursor.execute("""SELECT *
FROM (
SELECT DISTINCT ON (fieldin.playerid) fieldin.yearid, fieldin.playerid
FROM fieldin
GROUP BY fieldin.playerid, fieldin.yearid
ORDER BY fieldin.playerid, fieldin.yearid
) a

JOIN  (
SELECT DISTINCT ON (fieldin.playerid) fieldin.yearid, fieldin.playerid
FROM fieldin
GROUP BY fieldin.playerid, fieldin.yearid
ORDER BY fieldin.playerid, fieldin.yearid DESC
) b
ON a.playerid = b.playerid;""")

In [88]:
cursor.fetchone()

(2004, 'aardsda01', 2013, 'aardsda01')

3. Who has played the most all star games?

In [92]:
cursor.execute("""SELECT allstarfull.playerid, COUNT(*) as allstar_game_count
FROM allstarfull
GROUP BY allstarfull.playerid
ORDER BY allstar_game_count DESC
LIMIT 1;""")

In [93]:
cursor.fetchone()

('aaronha01', 25)

4. Which school has generated the most distinct players? *Hint:* Create new table from 'CollegePlaying.csv'.

In [114]:
cursor.execute("""CREATE TABLE IF NOT EXISTS SchoolsPlayer(
playerID varchar(20) NOT NULL,
schoolID text NOT NULL,
yearMin int DEFAULT NULL,
yearMax int DEFAULT NULL,
PRIMARY KEY (playerID, schoolID)
);""")

In [115]:
cursor.execute("COPY SchoolsPlayer FROM '/home/ubuntu/baseballdata/SchoolsPlayers.csv' DELIMITER ',' CSV HEADER;")
connection.commit()

In [117]:
cursor.execute("""SELECT schoolsplayer.schoolid, schools.schoolid, schools.schoolname, COUNT(DISTINCT schoolsplayer.playerid) as distinct_player_count
FROM schools
JOIN schoolsplayer
ON schools.schoolid = schoolsplayer.schoolid
GROUP BY schoolsplayer.schoolid, schools.schoolid
ORDER BY distinct_player_count desc
LIMIT 1;""")


In [118]:
cursor.fetchone()

('usc', 'usc', 'University of Southern California', 102)

5. Which players have the longest career? Assume that the `debut` and `finalGame` columns comprise the start and end, respectively, of a player's career.

In [136]:
cursor.execute("""CREATE TABLE IF NOT EXISTS player(
playerID varchar(20) NOT NULL, 
birthYear text DEFAULT NULL,
birthMonth int DEFAULT NULL,
birthDay int DEFAULT NULL,
birthCountry text DEFAULT NULL,
birthState text DEFAULT NULL,
birthCity text DEFAULT NULL,
deathYear int DEFAULT NULL,
deathMonth int DEFAULT NULL,
deathDay int DEFAULT NULL,
deathCountry text DEFAULT NULL,
deathState text DEFAULT NULL,
deathCity text DEFAULT NULL,
nameFirst text DEFAULT NULL,
nameLast text DEFAULT NULL,
nameGiven text DEFAULT NULL,
weight int DEFAULT NULL,
height int DEFAULT NULL,
bats text DEFAULT NULL,
throws text DEFAULT NULL,
debut text DEFAULT NULL,
finalGame text DEFAULT NULL,
retroID varchar(20) DEFAULT NULL,
bbrefIDnameGiven varchar(20) DEFAULT NULL,
PRIMARY KEY(playerID));""")




cursor.execute("""COPY player FROM '/home/ubuntu/baseballdata/Master.csv' DELIMITER ',' CSV HEADER;""")
connection.commit()

NotNullViolation: null value in column "namegiven" violates not-null constraint
DETAIL:  Failing row contains (bolan01, null, null, null, null, null, null, null, null, null, null, null, null, null, Boland, null, null, null, null, null, 1875-09-04, 1875-09-04, bolau101, bolan01).
CONTEXT:  COPY player, line 1457: "bolan01,,,,,,,,,,,,,,Boland,,,,,,1875-09-04,1875-09-04,bolau101,bolan01"


In [139]:
cursor.execute("""SELECT playerid, nameGiven, days_played
FROM (
SELECT player.playerid, player,nameGiven, to_date(player.finalGame, 'YYYY MM DD') - to_date(player.debut, 'YYYY MM DD') as days_played
FROM player) AS player_longevity
WHERE days_played > 0
ORDER BY days_played desc;""")

In [140]:
cursor.fetchone()

[('altroni01', 'Nicholas', 12862)]

6. What is the distribution of debut months?

In [ ]:
cursor.execute("""SELECT EXTRACT(MONTH FROM to_timestamp(player.debut, 'YYYY MM DD')) as month, COUNT(*) as debut_month_counts
FROM player
GROUP BY EXTRACT(MONTH FROM to_timestamp(player.debut, 'YYYY MM DD'))
ORDER BY EXTRACT(MONTH FROM to_timestamp(player.debut, 'YYYY MM DD'));""")

7. What is the effect of table join order on mean salary for the players listed in the main (master) table?

NO EFFECT. BOTH MERGES PRODUCED THE SAME MEAN SALARIES FOR THE PLAYERS.

In [ ]:
cursor.execute("""SELECT player.playerid, player.nameGiven, salaries.playerid, SUM(salaries.salary)/COUNT(*) as mean_salary_j_salaries
FROM player
LEFT JOIN salaries
ON player.playerid = salaries.playerid
GROUP BY player.playerid, salaries.playerid
ORDER BY player.playerid;""")

In [ ]:
cursor.fetchone()

In [ ]:
cursor.execute("""SELECT salaries.playerid, SUM(salaries.salary)/COUNT(*) as mean_salary_j_player, player.playerid,player.nameGiven
FROM salaries
LEFT JOIN player
ON salaries.playerid = player.playerid
GROUP BY salaries.playerid, player.playerid
ORDER BY salaries.playerid;""")

In [ ]:
cursor.fetchone()

In [ ]:
connection.commit()
connection.close()